In [1]:
import pandas as pd
import pickle as pk
from ckip_transformers.nlp import CkipNerChunker

import os
import json
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

/home/yc980802/.pyenv/versions/ptt/lib/python3.8/site-packages/pandas/compat/__init__.py:124: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [2]:
with open('info.pkl', 'rb') as f:
    info_dicts = pk.load(f)

In [3]:
urls, infos = info_dicts.keys(), info_dicts.values()
title_list = [info['title'] for info in infos]

In [4]:
ner_driver = CkipNerChunker(level=3, device=0)
ner = ner_driver(title_list, batch_size=1024, use_delim=True)

Inference: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 155/155 [02:12<00:00,  1.17it/s]


In [22]:
data = []
for url, sentence_ner in zip(urls, ner):
    # sentence_ner有兩個以上的專有名詞
    for entity in sentence_ner:
        if entity.ner == 'GPE':
            data.append([url, entity.word])

In [23]:
df = pd.DataFrame(data, columns=['url', 'country'])
inference_country = list(df.country.unique())
famous_country = pd.read_csv('country.csv').iloc[:, 0]

for fctry in famous_country:
    for ctry in inference_country:
        if fctry in ctry:
            df['country'].replace(ctry, fctry, inplace=True)
            
df = df[df['country'].isin(famous_country)]

In [24]:
jsondict = df.groupby('country')['url'].apply(list).to_dict()
needed_info = ['title']
for country, url_list in jsondict.items():
    content_list = []
    for url in url_list:
        info = {k: info_dicts[url][k] for k in needed_info}
        info['url'] = url
        content_list.append(info)
    jsondict[country] = content_list

In [25]:
with open('content.json', 'w') as jsonfile:
    json.dump(jsondict, jsonfile)